# Extracting Data from Job Postings - Data Cleaning

## Data Set Structure

1. Year - Taken from manually from the file name.

2. Job Type - Best to call openAI API and classify the job titles into 5 categories that we have defined.
    - ML Engineer
    - Data Analyst
    - Data Engineer
    - Data Science
    - Applied Reaserch 

        
2. Industry - The classifier below matches job description to industry (Predefined set of 67).
    - https://huggingface.co/sampathkethineedi/industry-classification

    
3. Technical Skills - for example data cleaning, feature engineering, etc.
    - https://huggingface.co/algiraldohe/lm-ner-linkedin-skills-recognition
    - The model works pretty well, has skill types
    - Doing this through OpenAI sounds like pain but idk


4. Technology skills - Python, SQL, Java, etc.
    - Same classification method as for Technical skills


5. Soft Skills - fast learner, bla bla bla corpo talk, etc.
    - Same classification as for Technology and Technical skills.


6. Salary 
    - Could use chatGPT for this, have not found a ready made solution. Prompting for this might be a pain since in my opinion ChatGPT is terrible with numbers.


7. Company Name
    - What if in one dataset we have Microsoft and in other Microsoft, inc?

7. Education
    - I am doing a regex matching function

8. Years of experience
    - Use OpenAi idk how to do it otherwise


9. City
    - https://huggingface.co/ml6team/bert-base-uncased-city-country-ner?text=Sunnyvale
    - 2023 dataset does not have this


10. State
    - 2023 dataset does not have this
    

# PLZ review and see what else is needed



## To run the code below make sure you have pyTorch and TensorFlow installed

`pip3 install torch torchvision torchaudio`

`pip install tensorflow `
    

In [32]:
#Imports
from transformers import pipeline
import re
import pandas as pd

In [77]:
#Skill Extraction
extract_skills = pipeline("token-classification", model="algiraldohe/lm-ner-linkedin-skills-recognition")

In [35]:
#Industry Classification
get_industry = pipeline("text-classification", model="sampathkethineedi/industry-classification-api")

In [36]:
# Use a pipeline as a high-level helper
get_city = pipeline("token-classification", model="ml6team/bert-base-uncased-city-country-ner")

In [108]:
#this function is wrong, if you input "Atlanta, GA", it will give you [LA, GA]
def find_state_codes(input_string):
    state_code_pattern = "([Aa][EeLlKkSsZzRr]|[Cc][AaOoTt]|[Dd][EeCc]|[Ff][MmLl]|[Gg][AaUu]|[Hh][Ii]|[Ii][DdLlNnAa]|[Kk][SsYy]|[Ll][Aa]|[Mm][EeHhDdAaIiNnSsOoTt]|[Nn][EeVvHhJjMmYyCcDd]|[Mm][Pp]|[Oo][HhKkRr]|[Pp][WwAaRr]|[Rr][Ii]|[Ss][CcDd]|[Tt][NnXx]|[Uu][Tt]|[Vv][TtIiAa]|[Ww][AaVvIiYy])"
    matches = re.findall(state_code_pattern, input_string, flags=re.IGNORECASE)
    return [match.upper() for match in matches]

In [111]:
#Degree Matching

def needs_bachelors_degree(job_posting):
    degree_keywords = ['Bachelor', 'BA', 'BS', 'BSc', 'B.A.', 'B.S.', 'B.Sc.', 'Undergraduate', 'AB', 'S.B.', 'SB', 'BEng', 'BFA', 'BBA', 'LLB', 'BCom', 'BEd', 'BMath', 'BArch', 'B.Tech', 'B.Des', 'B.Pharm', 'B.Nursing', 'B.HSc', 'B.IT', 'B.Econ']

    degree_pattern = re.compile(r'\b(?:' + '|'.join(degree_keywords) + r')\b', re.IGNORECASE)

    return bool(degree_pattern.search(job_posting))

def needs_masters_degree(job_posting):
    degree_keywords = ['Master', 'MS', 'MSc', 'M.A.', 'M.S.', 'M.Sc.', 'Graduate', 'MA', 'MEng', 'MFA', 'MBA', 'LLM', 'MCom', 'MEd', 'MPhil', 'MRes', 'MArch', 'M.Tech', 'M.Des', 'M.Pharm', 'M.Nursing', 'M.HSc', 'M.IT', 'M.Econ']

    degree_pattern = re.compile(r'\b(?:' + '|'.join(degree_keywords) + r')\b', re.IGNORECASE)

    return bool(degree_pattern.search(job_posting))

def needs_phd(job_posting):
    degree_keywords = ['PhD', 'Doctorate', 'Doctoral', 'Ph.D.', 'D.Phil', 'Dr.', 'DSc', 'DLitt', 'ThD', 'D.Tech', 'PharmD', 'ScD', 'EdD']

    degree_pattern = re.compile(r'\b(?:' + '|'.join(degree_keywords) + r')\b', re.IGNORECASE)

    return bool(degree_pattern.search(job_posting))

## Clean Data set 2019

In [122]:
#Read in Data set

csv_file_path = 'path/to/your/file.csv'

df2019 = pd.read_csv('job_posting_data/Data_Jobs_2019.csv')

df2019

,position,company,description,reviews,location
0,Development Director,ALS TDI,Development Director\nALS Therapy Development ...,NaN,"Atlanta, GA 30301"
1,An Ostentatiously-Excitable Principal Research...,The Hexagon Lavish,"Job Description\n\n""The road that leads to acc...",NaN,"Atlanta, GA"
2,Data Scientist,Xpert Staffing,"Growing company located in the Atlanta, GA are...",NaN,"Atlanta, GA"
3,Data Analyst,Operation HOPE,DEPARTMENT: Program OperationsPOSITION LOCATIO...,44.0,"Atlanta, GA 30303"
4,Assistant Professor -TT - Signal Processing & ...,Emory University,DESCRIPTION\nThe Emory University Department o...,550.0,"Atlanta, GA"
...,...,...,...,...,...
6959,Data Developer / Machine Learning Analyst,NetApp,Are you data-driven? We at NetApp believe in t...,574.0,"Sunnyvale, CA"
6960,Scientist I,"Pharmacyclics, an Abbvie Company",Pharmacyclics is committed to the development ...,26.0,"Sunnyvale, CA"
6961,Intern Scientist,Oath Inc,"Oath, a subsidiary of Verizon, is a values-led...",5.0,"Sunnyvale, CA"
6962,Senior Data & Applied Scientist,Microsoft,We are the Bing Core Relevance team responsibl...,4618.0,"Sunnyvale, CA"


In [123]:
#This dataset took like 5 minutes on my computer to process so if you want to see what processing does uncomment line below
df2019 = df2019.head()

df2019 = df2019.drop('reviews', axis=1)
df2019.rename(columns={'position': 'Job Type'}, inplace=True)
df2019.rename(columns={'company': 'Company Name'}, inplace=True)

def getIndustryFromDesc(x):
    if type(x) == str: #some job description was a float 
        industry_classification = get_industry(x[0:512]) #some pytorch function only allows strings up to 512 chars, but industry is always mentioned first so we good;
        return industry_classification[0].get("label"); # do we wanna check confidence intervals or fuck it?
    else:
       return ''

df2019['Industry'] = df2019['description'].apply(getIndustryFromDesc)

##TODO write a function so skills can be extracted correctly
def getSkillsFromDesc(x):
    dupa = extract_skills(x[0:512]) #TODO batch processing strings are limited to 512 chars
    #print(dupa)
    return 0 # you want to return skills based on category, b-bus, b-technical, the link is above in markdown

df2019['Skills'] = df2019['description'].apply(getSkillsFromDesc)

def getCityFromLocation(x):
    y = get_city(x);
    print(y);
    city = y[0].get('word');
    return city[0].upper() + city[1:];

df2019['City'] = df2019['location'].apply(getCityFromLocation)

df2019


[{'entity': 'CITY', 'score': 0.9728621, 'index': 1, 'word': 'atlanta', 'start': 0, 'end': 7}]
[{'entity': 'CITY', 'score': 0.9683732, 'index': 1, 'word': 'atlanta', 'start': 0, 'end': 7}]
[{'entity': 'CITY', 'score': 0.9683732, 'index': 1, 'word': 'atlanta', 'start': 0, 'end': 7}]
[{'entity': 'CITY', 'score': 0.97546536, 'index': 1, 'word': 'atlanta', 'start': 0, 'end': 7}]
[{'entity': 'CITY', 'score': 0.9683732, 'index': 1, 'word': 'atlanta', 'start': 0, 'end': 7}]


,Job Type,Company Name,description,location,Industry,Skills,City
0,Development Director,ALS TDI,Development Director\nALS Therapy Development ...,"Atlanta, GA 30301",Health Care Services,0,Atlanta
1,An Ostentatiously-Excitable Principal Research...,The Hexagon Lavish,"Job Description\n\n""The road that leads to acc...","Atlanta, GA",Human Resource & Employment Services,0,Atlanta
2,Data Scientist,Xpert Staffing,"Growing company located in the Atlanta, GA are...","Atlanta, GA",Life Sciences Tools & Services,0,Atlanta
3,Data Analyst,Operation HOPE,DEPARTMENT: Program OperationsPOSITION LOCATIO...,"Atlanta, GA 30303",Environmental & Facilities Services,0,Atlanta
4,Assistant Professor -TT - Signal Processing & ...,Emory University,DESCRIPTION\nThe Emory University Department o...,"Atlanta, GA",Life Sciences Tools & Services,0,Atlanta


In [22]:
## put it here to gauge how much using openAI would cost

import csv

def count_words_in_csv(file_path):
    word_count = 0

    with open(file_path, 'r') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            for column in row:
                words = column.split()
                word_count += len(words)

    return word_count

# Example usage:
file_path = 'your_csv_file.csv'
result = count_words_in_csv(file_path)
print(f'The number of words in the CSV file is: {result}')


FileNotFoundError: [Errno 2] No such file or directory: 'your_csv_file.csv'